### Use CNNs to Classify Hand-written Digits on MNIST Dataset

In [1]:
!pip install keras

In [2]:
# MNIST comes with Keras by default, so load simply
from keras.datasets import mnist

# loading the dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# let's print the shape of the dataset

In [4]:
print("X_train shape", X_train.shape)

X_train shape (60000, 28, 28)


## Before we train a CNN model, let’s build a basic Fully Connected Neural Network for the dataset

### To build an image classification model using a neural network:

- Flatten the input image dimensions to 1D (width pixels x height pixels)
- Normalize the image pixel values (divide by 255)
- One-Hot Encode the categorical column
- Build a model architecture (Sequential) with Dense layers
- Train the model and make predictions

### Step #0: keras imports for the dataset and building our neural network

In [5]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D
from keras.utils import np_utils

### Step #1: Flattening the images from the 28x28 pixels to 1D 784 pixels

In [6]:
X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

### Step #2: normalizing the data to help with the training

In [7]:
X_train /= 255
X_test /= 255

### Step #3: One-Hot Encode the categorical column

In [8]:
n_classes = 10
print("Shape before one-hot encoding: ", y_train.shape)
Y_train = np_utils.to_categorical(y_train, n_classes)
Y_test = np_utils.to_categorical(y_test, n_classes)
print("Shape after one-hot encoding: ", Y_train.shape)

Shape before one-hot encoding:  (60000,)
Shape after one-hot encoding:  (60000, 10)


In [9]:
y_train[0:5]

array([5, 0, 4, 1, 9], dtype=uint8)

In [10]:
Y_train[0:5,]

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]], dtype=float32)

### Step #4: building a linear stack of layers with the sequential model

In [11]:
model = Sequential()
# hidden layer
model.add(Dense(100, input_shape=(784,), activation='relu'))
# output layer
model.add(Dense(10, activation='softmax'))

2023-04-06 19:10:46.207021: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-06 19:10:46.266772: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fba6c231b80 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2023-04-06 19:10:46.266788: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version


### Step #5: Train the model 

In [12]:
# looking at the model summary
model.summary()
# compiling the sequential model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
# training the model for 10 epochs
model.fit(X_train, Y_train, batch_size=128, epochs=10, validation_data=(X_test, Y_test))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               78500     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1010      
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
469/469 [==============================] - 1s 1ms/step - loss: 0.3803 - accuracy: 0.8946 - val_loss: 0.2119 - val_accuracy: 0.9384
Epoch 2/10
469/469 [==============================] - 0s 1ms/step - loss: 0.1758 - accuracy: 0.9489 - val_loss: 0.1484 - val_accuracy: 0.9578
Epoch 3/10
469/469 [==============================] - 0s 1ms/step - loss: 0.1280 - accuracy: 0.9632 - val_loss: 0.1180 - val_accuracy: 0.9644
Epoch 4/10
469/469 [==============================] - 0s 1ms/step - loss: 0.0997 - accur

### Let’s modify the above code to build a CNN model 
One major advantage of using CNNs over NNs is that you do not need to flatten the input images to 1D as they are capable of working with image data in 2D. This helps in retaining the “spatial” properties of images.


In [13]:
# keras imports for the dataset and building our neural network
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
from keras.utils import np_utils

# to calculate accuracy
from sklearn.metrics import accuracy_score

# loading the dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# building the input vector from the 28x28 pixels
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# normalizing the data to help with the training
X_train /= 255
X_test /= 255

# one-hot encoding using keras' numpy-related utilities
n_classes = 10
print("Shape before one-hot encoding: ", y_train.shape)
Y_train = np_utils.to_categorical(y_train, n_classes)
Y_test = np_utils.to_categorical(y_test, n_classes)
print("Shape after one-hot encoding: ", Y_train.shape)

# building a linear stack of layers with the sequential model
model = Sequential()
# convolutional layer
model.add(Conv2D(25, kernel_size=(3,3), strides=(1,1), padding='valid', activation='relu', input_shape=(28,28,1)))
model.add(MaxPool2D(pool_size=(1,1)))
# flatten output of conv
model.add(Flatten())
# hidden layer
model.add(Dense(100, activation='relu'))
# output layer
model.add(Dense(10, activation='softmax'))

# compiling the sequential model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

# training the model for 10 epochs
model.fit(X_train, Y_train, batch_size=128, epochs=10, validation_data=(X_test, Y_test))


Shape before one-hot encoding:  (60000,)
Shape after one-hot encoding:  (60000, 10)
Epoch 1/10
469/469 [==============================] - 15s 32ms/step - loss: 0.2006 - accuracy: 0.9401 - val_loss: 0.0757 - val_accuracy: 0.9766
Epoch 2/10
469/469 [==============================] - 15s 31ms/step - loss: 0.0607 - accuracy: 0.9823 - val_loss: 0.0589 - val_accuracy: 0.9823
Epoch 3/10
469/469 [==============================] - 15s 32ms/step - loss: 0.0383 - accuracy: 0.9886 - val_loss: 0.0527 - val_accuracy: 0.9840
Epoch 4/10
469/469 [==============================] - 15s 32ms/step - loss: 0.0258 - accuracy: 0.9918 - val_loss: 0.0507 - val_accuracy: 0.9845
Epoch 5/10
469/469 [==============================] - 15s 32ms/step - loss: 0.0187 - accuracy: 0.9942 - val_loss: 0.0524 - val_accuracy: 0.9836
Epoch 6/10
469/469 [==============================] - 16s 34ms/step - loss: 0.0106 - accuracy: 0.9970 - val_loss: 0.0632 - val_accuracy: 0.9823
Epoch 7/10
469/469 [==============================] 

### Conclusion: 

* Simple neural network model was around 94% (82%), the CNN model is able to get 98%+ with just a single convolution layer
* You can go ahead and add more Conv2D layers, and also play around with the hyperparameters of the CNN model.


### TO-DO 
1. Try to visulize images

2. Try to classfiy images from the CIFAR-10 Dataset using CNNs https://www.cs.toronto.edu/~kriz/cifar.html. 
The CIFAR-10 dataset consists of 60,000 32 x 32 colour images in 10 classes, with 6,000 images per class. There are 50,000 training images and 10,000 test images.

3. Try to Categorize the Images of ImageNet using CNNs (http://www.image-net.org/). The ImageNet dataset has more than 14 million images, hand-labeled across 20,000 categories.

4. Try to track learning curve